### Import Libraries

In [ ]:
!pip install mlrun

     |████████████████████████████████| 460kB 6.5MB/s 
     |████████████████████████████████| 17.3MB 175kB/s 
     |████████████████████████████████| 665kB 40.4MB/s 
     |████████████████████████████████| 10.1MB 26.8MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 645kB 34.7MB/s 
     |████████████████████████████████| 174kB 43.9MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 1.5MB 33.9MB/s 
     |████████████████████████████████| 2.6MB 38.1MB/s 
     |████████████████████████████████| 215kB 52.4MB/s 
     |████████████████████████████████| 143kB 55.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 122kB 53.0MB/s 
     |████████████████████████████████| 10.8MB 20.4MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 51kB 4.9MB/s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from mlrun import mlconf

mlconf.dbpath = '/content/drive/MyDrive/Hackathon'
mlconf.artifact_path = '/content/drive/MyDrive/Hackathon/Data'

In [ ]:
from os import path
import mlrun

project_name_base = 'suicide-pred'

project_name, artifact_path = mlrun.set_environment(project=project_name_base, user_project=True)

print(f'Project name: {project_name}')
print(f'Artifact path: {artifact_path}')

Project name: suicide-pred-root
Artifact path: /content/drive/MyDrive/Hackathon/Data


<h2>MLRun Functions

In [ ]:
# nuclio: start-code

In [ ]:
!pip install pymongo[srv]

import pickle
from pickle import dumps
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
from os import path
import os
import mlrun

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

# nltk.download()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from pymongo import MongoClient
import urllib
import sys
import pandas as pd
import pymongo
import json
import os

def read_content():
    m_client = pymongo.MongoClient("mongodb+srv://harsh:<password>@cluster0.hh8w7.mongodb.net/<dbname>?retryWrites=true&w=majority")
    db = m_client.test


    #m_client = pymongo.MongoClient("mongodb://...")
    m_db = m_client["suicide"]
    db_cm = m_db["suicide"]
    df = pd.DataFrame.from_records(db_cm.find())
    print('--------------------------------')
    print()
    print(df)
    return df

In [ ]:
df=read_content()

--------------------------------

                           _id  ...                                              tweet
0     60e08fddeee7fb8ea50f3d85  ...  my life is meaningless i just want to end my l...
1     60e08fdfeee7fb8ea50f3d86  ...  muttering i wanna die to myself daily for a fe...
2     60e08fdfeee7fb8ea50f3d87  ...  work slave i really feel like my only purpose ...
3     60e08fe0eee7fb8ea50f3d88  ...  i did something on the 2 of october i overdose...
4     60e08fe0eee7fb8ea50f3d89  ...  i feel like no one cares i just want to die ma...
...                        ...  ...                                                ...
9114  60e09986eee7fb8ea50f611f  ...  have you ever laid on your bed at night and cr...
9115  60e09987eee7fb8ea50f6120  ...  the fault the blame the pain s still there i m...
9116  60e09987eee7fb8ea50f6121  ...  stop asking me to trust you when i m still cou...
9117  60e09987eee7fb8ea50f6122  ...  i never know how to handle sadness crying make...
9118  60e

In [ ]:
df=df[['label','tweet']]

In [ ]:
df

,label,tweet
0,1,my life is meaningless i just want to end my l...
1,1,muttering i wanna die to myself daily for a fe...
2,1,work slave i really feel like my only purpose ...
3,1,i did something on the 2 of october i overdose...
4,1,i feel like no one cares i just want to die ma...
...,...,...
9114,1,have you ever laid on your bed at night and cr...
9115,1,the fault the blame the pain s still there i m...
9116,1,stop asking me to trust you when i m still cou...
9117,1,i never know how to handle sadness crying make...


In [ ]:
df.to_csv('/content/drive/MyDrive/Hackathon/suicidal_data1.csv')

In [ ]:
def fetch_data(context : MLClientCtx, data_path: DataItem):
    
    context.logger.info('Reading data from {}'.format(data_path))

    suicide_dataset = df
    
    
    target_path = path.join(context.artifact_path, 'data')
    context.logger.info('Saving datasets to {} ...'.format(target_path))

    # Store the data sets in your artifacts database
    context.log_dataset('suicide_dataset', df=suicide_dataset, format='csv',
                        index=False, artifact_path=target_path)
    

In [ ]:
def preprocess_tweet(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text+' '.join(emoticons).replace('-', '') 
    return text

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text += ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in tokenizer_porter(text) if w not in stop]
    return tokenized

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, 
                         preprocessor=None,tokenizer=tokenizer)

In [ ]:
def transform_dataset(context : MLClientCtx, data: DataItem):

  context.logger.info('Begin datasets transform')

  df = data.as_df()
  df['tweet'] = df['tweet'].apply(lambda x: preprocess_tweet(x))
  # print(len(df["tweet"].to_list()))
  # k = list(vect.transform(df["tweet"].values))
  # print(len(k))
  # df['tweet'] = list(vect.transform(df["tweet"].values))

  target_path = path.join(context.artifact_path, 'data')
  context.log_dataset('suicide_dataset_transformed', df=df, artifact_path=target_path, format='csv')    


In [ ]:
def train_model(context: MLClientCtx, input_ds: DataItem):


  context.logger.info('Begin training')
  from sklearn.linear_model import SGDClassifier
  clf = SGDClassifier(loss='log', random_state=1)

  df = input_ds.as_df()
  X = df["tweet"].to_list()
  y = df['label']

  from sklearn.model_selection import train_test_split
  X_train,X_test,y_train,y_test = train_test_split(X,
                                                  y,
                                                  test_size=0.20,
                                                  random_state=0)  
  X_train = vect.transform(X_train)
  X_test = vect.transform(X_test)

  classes = np.array([0, 1])
  clf.partial_fit(X_train, y_train,classes=classes)

  print('Accuracy: %.3f' % clf.score(X_test, y_test))

  context.log_model('Suicide_Model',
                     body=dumps(clf),
                     artifact_path=context.artifact_subpath("models"),
                     model_file="Suicide_Model.pkl")
  

  context.logger.info('End training')

In [ ]:
import numpy as np
from cloudpickle import load

class SuicideModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = body['inputs'][0]
            # feats = feats.decode('ISO-8859-1')
            feats = preprocess_tweet(feats)
            l = []
            l.append(feats)
            feats = vect.transform(l)
            print('hi')
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [ ]:
# nuclio: end-code

<h2>MLRun Procedure 

In [ ]:
suicide_func = mlrun.code_to_function(name='suicide', kind='job', filename = '/content/drive/MyDrive/Colab Notebooks/code_MLRun.ipynb')

<h3> Fetch

In [ ]:
fetch_data_run = suicide_func.run(handler='fetch_data',
                               inputs={'data_path': '/content/drive/MyDrive/Hackathon/suicidal_data1.csv'},
                               local=True)

> 2021-07-04 05:24:50,048 [info] starting run suicide-fetch_data uid=f12b1ff1fedd47a688ccc8d829aad4c0 DB=/content/drive/MyDrive/Hackathon
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--------------------------------

                           _id  ...                                              tweet
0     60e08fddeee7fb8ea50f3d85  ...  my life is meaningless i just want to end my l...
1     60e08fdfeee7fb8ea50f3d86  ...  muttering i wanna die to myself daily for a fe...
2     60e08fdfeee7fb8ea50f3d87  ...  work slave i really feel like my only purpose ...
3     60e08fe0eee7fb8ea50f3d88  ...  i did something on the 2 of october i overdose...
4     60e08fe0eee7fb8ea50f3d89  ...  i feel like no one cares i just want to die ma...
...                        ...  ...                                                ...
9114  60e09986eee7fb8ea50f611f  ...  have you ever laid on your bed at night and cr...
9115  60e0998

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
suicide-pred-root,...aad4c0,0,Jul 04 05:24:51,completed,suicide-fetch_data,kind=owner=roothost=ed438d8ab165,data_path,,,suicide_dataset


to track results use .show() or .logs() or in CLI: 
!mlrun get run f12b1ff1fedd47a688ccc8d829aad4c0 --project suicide-pred-root , !mlrun logs f12b1ff1fedd47a688ccc8d829aad4c0 --project suicide-pred-root
> 2021-07-04 05:24:52,829 [info] run executed, status=completed


In [ ]:
fetch_data_run.outputs

{'suicide_dataset': 'store://artifacts/suicide-pred-root/suicide-fetch_data_suicide_dataset:f12b1ff1fedd47a688ccc8d829aad4c0'}

<h3> Transform

In [ ]:
transform_dataset_run = suicide_func.run(name='transform_dataset',
                                      handler='transform_dataset',
                                      inputs={'data': fetch_data_run.outputs['suicide_dataset']},
                                      local=True)

> 2021-07-04 05:24:56,386 [info] starting run transform_dataset uid=1d9744cc85b546e5bc01940d76f06752 DB=/content/drive/MyDrive/Hackathon
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--------------------------------

                           _id  ...                                              tweet
0     60e08fddeee7fb8ea50f3d85  ...  my life is meaningless i just want to end my l...
1     60e08fdfeee7fb8ea50f3d86  ...  muttering i wanna die to myself daily for a fe...
2     60e08fdfeee7fb8ea50f3d87  ...  work slave i really feel like my only purpose ...
3     60e08fe0eee7fb8ea50f3d88  ...  i did something on the 2 of october i overdose...
4     60e08fe0eee7fb8ea50f3d89  ...  i feel like no one cares i just want to die ma...
...                        ...  ...                                                ...
9114  60e09986eee7fb8ea50f611f  ...  have you ever laid on your bed at night and cr...
9115  60e09987

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
suicide-pred-root,...f06752,0,Jul 04 05:24:57,completed,transform_dataset,kind=owner=roothost=ed438d8ab165,data,,,suicide_dataset_transformed


to track results use .show() or .logs() or in CLI: 
!mlrun get run 1d9744cc85b546e5bc01940d76f06752 --project suicide-pred-root , !mlrun logs 1d9744cc85b546e5bc01940d76f06752 --project suicide-pred-root
> 2021-07-04 05:24:58,924 [info] run executed, status=completed


In [ ]:
transform_dataset_run.outputs

{'suicide_dataset_transformed': 'store://artifacts/suicide-pred-root/transform_dataset_suicide_dataset_transformed:1d9744cc85b546e5bc01940d76f06752'}

<h3> Train

In [ ]:
train_model_run = suicide_func.run(name='train_model',
                                handler='train_model',
                                inputs={'input_ds': transform_dataset_run.outputs['suicide_dataset_transformed']},
                                local=True)

> 2021-07-04 05:25:03,663 [info] starting run train_model uid=244e390cdc784189b698a6e39a9edc00 DB=/content/drive/MyDrive/Hackathon
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--------------------------------

                           _id  ...                                              tweet
0     60e08fddeee7fb8ea50f3d85  ...  my life is meaningless i just want to end my l...
1     60e08fdfeee7fb8ea50f3d86  ...  muttering i wanna die to myself daily for a fe...
2     60e08fdfeee7fb8ea50f3d87  ...  work slave i really feel like my only purpose ...
3     60e08fe0eee7fb8ea50f3d88  ...  i did something on the 2 of october i overdose...
4     60e08fe0eee7fb8ea50f3d89  ...  i feel like no one cares i just want to die ma...
...                        ...  ...                                                ...
9114  60e09986eee7fb8ea50f611f  ...  have you ever laid on your bed at night and cr...
9115  60e09987eee7fb

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
suicide-pred-root,...9edc00,0,Jul 04 05:25:04,completed,train_model,kind=owner=roothost=ed438d8ab165,input_ds,,,Suicide_Model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 244e390cdc784189b698a6e39a9edc00 --project suicide-pred-root , !mlrun logs 244e390cdc784189b698a6e39a9edc00 --project suicide-pred-root
> 2021-07-04 05:25:21,223 [info] run executed, status=completed


In [ ]:
train_model_run.outputs

{'Suicide_Model': 'store://artifacts/suicide-pred-root/train_model_Suicide_Model:244e390cdc784189b698a6e39a9edc00'}

<h3> Serving 

In [ ]:
serving = mlrun.code_to_function('seving', filename='/content/drive/MyDrive/Colab Notebooks/code_MLRun.ipynb', kind='serving')

serving.spec.default_class = 'SuicideModel'
serving.add_model('suicide-serving', train_model_run.outputs['Suicide_Model'])
# serving_address = serving.deploy()

In [ ]:
my_data = '''{"inputs":["I'll kill myself am tired of living depressed and alone"]}'''

server = serving.to_mock_server()
server.test("/v2/models/suicide-serving/infer", body=my_data)

> 2021-07-04 05:25:57,142 [info] model suicide-serving was loaded
> 2021-07-04 05:25:57,144 [info] Loaded ['suicide-serving']
hi


{'id': '3df47f5ae47746108122ea73a3b1a3fd',
 'model_name': 'suicide-serving',
 'outputs': [1]}

In [ ]:
my_data = '''{"inputs":["It's such a hot day, I'd like to have ice cream and visit the park"]}'''

server = serving.to_mock_server()
server.test("/v2/models/suicide-serving/infer", body=my_data)

> 2021-07-04 05:25:59,923 [info] model suicide-serving was loaded
> 2021-07-04 05:25:59,925 [info] Loaded ['suicide-serving']
hi


{'id': 'e2bde7c755d84f24947f0787ac4ec59e',
 'model_name': 'suicide-serving',
 'outputs': [0]}